In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob


In [2]:

PATH = r"C:\Users\Dalia\Desktop\Data Science project\CovidTests"
EXT = "*.csv"
i=0
file_data = pd.DataFrame()
all_csv_files = []
for path, subdir, files in os.walk(PATH):
    for file in glob(os.path.join(path, EXT)):        
        file_name = pd.read_csv(file)        
        i=i+1
        if i>=1:
            file_data = file_data.append(file_name)
            print('number of rows after merging '+str(i)+' files: '+str(file_data['ct_e200_sukurimo_data'].count()))
        
print('total files: '+str(i))


number of rows after merging 1 files: 181569
number of rows after merging 2 files: 417236
number of rows after merging 3 files: 557401
number of rows after merging 4 files: 691942
total files: 4


In [3]:
#display(file_data.head())
#print(file_data['ct_paciento_amzius'].value_counts().sort_index())
#print(file_data['paciento_lytis'].value_counts().sort_index())
#print(file_data['ct_paciento_savivaldybe'].value_counts().sort_index())

In [11]:
print(file_data.shape)
file_data["Year"] = pd.DatetimeIndex(file_data['ct_e200_sukurimo_data']).year
file_data["Month"] = pd.DatetimeIndex(file_data['ct_e200_sukurimo_data']).month
file_data_adj = file_data[["Year","Month",'ct_e200_sukurimo_data','ct_paciento_amzius','paciento_lytis','ct_paciento_savivaldybe','testo_eiles_nr_asmeniui']].sort_values('ct_e200_sukurimo_data').reset_index(drop = True)

#display(file_data_adj.head())
#display(file_data_adj.tail())

(691942, 10)


In [5]:
file_data_adj= file_data_adj[file_data_adj['testo_eiles_nr_asmeniui']==1]
file_data_grouped = file_data_adj.groupby(['Year', 'Month','ct_paciento_amzius', 'paciento_lytis', 'ct_paciento_savivaldybe' ]).agg({'testo_eiles_nr_asmeniui':'count'})

Table_ready =  file_data_grouped.reset_index()

display(Table_ready.head())

,Year,Month,ct_paciento_amzius,paciento_lytis,ct_paciento_savivaldybe,testo_eiles_nr_asmeniui
0,2020,8,0-9,M,Alytaus m. sav.,20
1,2020,8,0-9,M,Alytaus r. sav.,5
2,2020,8,0-9,M,Anykščių r. sav.,3
3,2020,8,0-9,M,Biržų r. sav.,7
4,2020,8,0-9,M,Druskininkų sav.,9


In [6]:

# create a list of our conditions
conditions_prep = [
    (Table_ready['ct_paciento_amzius'].str.len()==2),
    (Table_ready['ct_paciento_amzius']=='Nenustatyta'),
    (Table_ready['ct_paciento_amzius'].str.len()>2)
    ]

# create a list of the values we want to assign for each condition
values_prep = ['number', 'NaN age', 'range']

# create a new column and use np.select to assign values to it using our lists as arguments
Table_ready['prep_age_range'] = np.select(conditions_prep, values_prep)

# create a list of our conditions
conditions_age_prep1 = [
    (Table_ready['prep_age_range']=='number'),
    (Table_ready['prep_age_range']=='range' ),
    (Table_ready['prep_age_range']=='NaN age' ),
    ]

# create a list of the values we want to assign for each condition
values_age_prep1 = [Table_ready['ct_paciento_amzius'], 0, 0 ]

# create a new column and use np.select to assign values to it using our lists as arguments
Table_ready['age_range_prep1'] = np.select(conditions_age_prep1, values_age_prep1)

# create a list of our conditions
conditions_age = [
    (Table_ready['prep_age_range']=='number') & (Table_ready['age_range_prep1'].astype(int) > 24) & (Table_ready['age_range_prep1'].astype(int) <= 30),
    (Table_ready['prep_age_range']=='number') & (Table_ready['age_range_prep1'].astype(int) > 30) & (Table_ready['age_range_prep1'].astype(int) <= 40),
    (Table_ready['prep_age_range']=='number') & (Table_ready['age_range_prep1'].astype(int) > 40) & (Table_ready['age_range_prep1'].astype(int) <= 50),
    (Table_ready['prep_age_range']=='number') & (Table_ready['age_range_prep1'].astype(int) > 50) & (Table_ready['age_range_prep1'].astype(int) <= 64),
    (Table_ready['prep_age_range']=='range' ),
    (Table_ready['prep_age_range']=='NaN age' ),
    ]

# create a list of the values we want to assign for each condition
values_age = ['25-30', '31-40', '41-50', '51-64', Table_ready['ct_paciento_amzius'].values,'unknown age' ]

# create a new column and use np.select to assign values to it using our lists as arguments
Table_ready['age_range'] = np.select(conditions_age, values_age)

Table_ready['paciento_lytis']=Table_ready['paciento_lytis'].replace(to_replace='Nenustatyta', value='unknown sex', regex=True)

# display updated DataFrame
display(Table_ready.head())

,Year,Month,ct_paciento_amzius,paciento_lytis,ct_paciento_savivaldybe,testo_eiles_nr_asmeniui,prep_age_range,age_range_prep1,age_range
0,2020,8,0-9,M,Alytaus m. sav.,20,range,0,0-9
1,2020,8,0-9,M,Alytaus r. sav.,5,range,0,0-9
2,2020,8,0-9,M,Anykščių r. sav.,3,range,0,0-9
3,2020,8,0-9,M,Biržų r. sav.,7,range,0,0-9
4,2020,8,0-9,M,Druskininkų sav.,9,range,0,0-9


In [19]:

T1 = Table_ready[['paciento_lytis','age_range','testo_eiles_nr_asmeniui']].groupby(['paciento_lytis','age_range']).agg({'testo_eiles_nr_asmeniui':'sum'}).reset_index()
T1=T1.sort_values('age_range').reset_index(drop = True)

T11 = Table_ready[['ct_paciento_savivaldybe', 'paciento_lytis','age_range','testo_eiles_nr_asmeniui']].groupby(['ct_paciento_savivaldybe','paciento_lytis', 'age_range']).agg({'testo_eiles_nr_asmeniui':'sum'}).reset_index()
T11_list = T11[['ct_paciento_savivaldybe', 'testo_eiles_nr_asmeniui']].groupby(['ct_paciento_savivaldybe']).agg({'testo_eiles_nr_asmeniui':'sum'}).reset_index()

T11_district_top6  = T11_list.nlargest(6, 'testo_eiles_nr_asmeniui')['ct_paciento_savivaldybe']

T111 = T11.merge(T11_district_top6, on = 'ct_paciento_savivaldybe', how = "outer", indicator = True)

conditions = [
    (T111['_merge']=="both"),
    (T111['_merge']=="left_only")
    ]

# create a list of the values we want to assign for each condition
values = [ T111['ct_paciento_savivaldybe'].values, 'Group of districts' ]

# create a new column and use np.select to assign values to it using our lists as arguments
T111['District'] = np.select(conditions, values)

T1 = T111[['District','age_range', 'testo_eiles_nr_asmeniui']].groupby(['District','age_range']).agg({'testo_eiles_nr_asmeniui':'sum'}).reset_index() 
T111_Grouped = T111[['paciento_lytis','District', 'testo_eiles_nr_asmeniui']].groupby(['paciento_lytis','District']).agg({'testo_eiles_nr_asmeniui':'sum'}).reset_index()

District_df = pd.DataFrame({'ct_paciento_savivaldybe':['Vilniaus m. sav.','Kauno m. sav.',
                                                       'Klaipėdos m. sav.','Šiaulių m. sav.',
                                                       'Panevėžio m. sav.','Nenustatyta']})
T111=T111.drop(columns = '_merge')
T111 = T111.merge(District_df, on = 'ct_paciento_savivaldybe', how = "outer", indicator = True)

conditions = [
    ((T111['_merge']=="both") & (T111['ct_paciento_savivaldybe']=='Nenustatyta')),
    ((T111['_merge']=="both") & (T111['ct_paciento_savivaldybe']!='Nenustatyta')),
    (T111['_merge']=="left_only")
    ]

# create a list of the values we want to assign for each condition
values = [ 'unknown district', T111['ct_paciento_savivaldybe'].values, 'Group of districts' ]

# create a new column and use np.select to assign values to it using our lists as arguments
T111['Top District'] = np.select(conditions, values)

T111_Top_Grouped = T111[['paciento_lytis','Top District', 'testo_eiles_nr_asmeniui']].groupby(['paciento_lytis','Top District']).agg({'testo_eiles_nr_asmeniui':'sum'}).reset_index()

T1_Top = T111[['Top District','age_range', 'testo_eiles_nr_asmeniui']].groupby(['Top District','age_range']).agg({'testo_eiles_nr_asmeniui':'sum'}).reset_index() 
display(T1_Top['Top District'].unique())

array(['Group of districts', 'Kauno m. sav.', 'Klaipėdos m. sav.',
       'Panevėžio m. sav.', 'Vilniaus m. sav.', 'unknown district',
       'Šiaulių m. sav.'], dtype=object)

In [21]:

# Node table

arr_sex = T111['paciento_lytis'].unique()
arr_age = T1['age_range'].unique()
arr_dist = T111['District'].unique()

arr_dist_top= T1_Top['Top District'].unique()
varcolors = [['pink',"#92b4f2",'#purple',
            '#763280', '#8d3c9a','#a546b3','#b35dbf','#bf77c9','#cb90d4','#d7aade',
              '#fdbf6f','#fdbf6f','#fdbf6f','#fdbf6f','#fdbf6f','#fdbf6f','#fdbf6f','#fdbf6f',
            '#fdbf6f','#fdbf6f','#fdbf6f','#fdbf6f','#fdbf6f']]
List = np.array(pd.DataFrame(np.concatenate((arr_sex,arr_dist_top, arr_age), axis=0).tolist()).reset_index())
ListT =np.transpose(List).tolist()

ListT.extend(varcolors)

nodes = np.transpose(ListT).tolist()

n_head = [['ID', 'Label', 'Color']]

display(nodes)


[['0', 'M', 'pink'],
 ['1', 'V', '#92b4f2'],
 ['2', 'unknown sex', '#purple'],
 ['3', 'Group of districts', '#763280'],
 ['4', 'Kauno m. sav.', '#8d3c9a'],
 ['5', 'Klaipėdos m. sav.', '#a546b3'],
 ['6', 'Panevėžio m. sav.', '#b35dbf'],
 ['7', 'Vilniaus m. sav.', '#bf77c9'],
 ['8', 'unknown district', '#cb90d4'],
 ['9', 'Šiaulių m. sav.', '#d7aade'],
 ['10', '0-9', '#fdbf6f'],
 ['11', '10-17', '#fdbf6f'],
 ['12', '18-24', '#fdbf6f'],
 ['13', '25-30', '#fdbf6f'],
 ['14', '31-40', '#fdbf6f'],
 ['15', '41-50', '#fdbf6f'],
 ['16', '51-64', '#fdbf6f'],
 ['17', '65-69', '#fdbf6f'],
 ['18', '70-74', '#fdbf6f'],
 ['19', '75-79', '#fdbf6f'],
 ['20', '80-84', '#fdbf6f'],
 ['21', '85+', '#fdbf6f'],
 ['22', 'unknown age', '#fdbf6f']]

In [22]:
import numpy as np

edges = np.concatenate((np.array(T111_Top_Grouped), np.array(T1_Top)), axis=0)

A = pd.DataFrame(edges, columns = ["A", "B", "C"])

conditions = [
    (A["A"]=="M"),
    (A["A"]=="V"),
    (A["A"]=="unknown sex"),
        (A["A"]=='Group of districts'),
        (A["A"]=='Vilniaus m. sav.'),
        (A["A"]=='Kauno m. sav.'),
        (A["A"]=='Klaipėdos m. sav.'),
        (A["A"]=='Šiaulių m. sav.'),
        (A["A"]=='Panevėžio m. sav.'),
        (A["A"]=='unknown district')
    
    ]

# create a list of the values we want to assign for each condition
values = [ "#fedcdb", "#c9daf9", "cab2d6",'#763280', '#8d3c9a','#a546b3','#b35dbf','#bf77c9','#cb90d4','#d7aade']

# create a new column and use np.select to assign values to it using our lists as arguments
A['color'] = np.select(conditions, values)

nodeL_df = pd.DataFrame(nodes, columns = ['ID', 'A', 'Color'])

A_adj  = A.merge(nodeL_df, on = "A", how = "left")
A_adj1  = A_adj.merge(nodeL_df, left_on = "B", right_on = "A", how = "left")
A_final = A_adj1[['ID_x','ID_y', 'C', 'color']]

edges_with_color = np.array(A_final)

e_head = [['Source','Target','Value','Link Color']]
links_without_head = edges_with_color.tolist()
#display(links_without_head)

In [24]:

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

nodes_df = pd.DataFrame(nodes, columns = ['ID', 'Label', 'Color'])


links_df = pd.DataFrame(links_without_head, columns =['Source','Target','Value','Link Color'] )


data_trace = dict(
    type='sankey',
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 25,
      thickness = 30,
      line = dict(
        color = "black",
        width = 0
      ),
      label =  nodes_df['Label'].dropna(axis=0, how='any'),
      color = nodes_df['Color']
    ),
    link = dict(
      source = links_df['Source'].dropna(axis=0, how='any'),
      target = links_df['Target'].dropna(axis=0, how='any'),
      value = links_df['Value'].dropna(axis=0, how='any'),
      color = links_df['Link Color'].dropna(axis=0, how='any'),
  )
)

layout =  dict(
    title = "Number of Covid-19 tests in Lithuania  ",
    height = 700,
    font = dict(
      size = 13
    ),    
)

fig = dict(data=[data_trace], layout=layout)
iplot(fig, validate=False)

